In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/real-and-fake-news-dataset/news.csv


In [2]:
# Initalizing our dataframe
df =  pd.read_csv('/kaggle/input/real-and-fake-news-dataset/news.csv')

# Display inital dataframe
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
# Importing necessary libraries and modules
!pip install nltk
import nltk
import random
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# NOTE: These libraries do not work when I use them
#from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
#lemmatizer = WordNetLemmatizer()


/opt/conda/lib/python3.10/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
# Preprocessing our 'text' column in our dataset
# I am only doing the text column because it is a better indicator of the true sentiment, rather than the title

# Function to be applied to every row in the 'text' column
def initializeDataset(row_text):

    # Tokenize the dataset and lowercase folding
    tokens = word_tokenize(row_text.lower())
    
    # Remove stop words
    tokensFiltered = [token for token in tokens if token not in stopwords.words('english')]
    
    # Lemmatizing each token (if the library worked for me)
    #tokens_lemmatized = lammatizer.lemmatize(token))
  
    # Adding the tokens back together
    initalized_text = ' '.join(tokensFiltered)
        
    # Returning the initalized text
    return initalized_text
    
# Preprocessing the dataset
df['text'] = df['text'].apply(initializeDataset)

# Displaying the first few rows in the preprocessed dataset
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"daniel greenfield , shillman journalism fellow...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,google pinterest digg linkedin reddit stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,u.s. secretary state john f. kerry said monday...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— kaydee king ( @ kaydeeking ) november 9 , 20...",FAKE
4,875,The Battle of New York: Why This Primary Matters,'s primary day new york front-runners hillary ...,REAL


In [5]:
# Defining our training and set documents

# Calculating the amount of rows in the dataframe
total_rows = len(df)

# Split the DataFrame into training and testing sets
training_set_rows = int(0.7 * total_rows)  # Setting the training set rows to 70%
train_df = df.iloc[:training_set_rows] # Training = 70% (0 - 70%)
test_df = df.iloc[training_set_rows:] # Testing = 30% (70% - 100%)

# Printing the Length of Train Dataframe (70%)
pd.set_option('display.max_colwidth', None)
print("Train DF:", len(train_df))

# Printing the Length of Test Dataframe (30%)
print("Test DF", len(test_df))

Train DF: 4434
Test DF 1901


In [6]:
# Doing Sentiment Analysis for the training Dataframe
# Each block within the training set is given a sentiment score (positive or negative)

# Instance of the Sentiment Analyzer
sentiment = SentimentIntensityAnalyzer()

# Getting a sentiment score for each row's 'text'
def get_sentiment_score(training_row):
    
    # Getting the Sentiment Score for the current sentence
    # Returns the format {'neg': X.X, 'neu': X.XXX, 'pos': X.XXX, 'compound': X.XXX}
    sentiment_score = sentiment.polarity_scores(training_row)

    #  Check which distribution it leans towards (positive or negative)
    if (sentiment_score['compound'] == 0):
        # Text is completely neutral (neither positive or negative)
        score_result = 0
    elif (sentiment_score['compound'] > 0):
        # Test is positive
        score_result = 1
    else:
        # Text is negative
        score_result = -1
    
    # Returning sentiment result
    return score_result

# Adding new "Sentinment" Column with the score for each row
train_df.loc[:, 'sentiment'] = train_df['text'].apply(get_sentiment_score)

# Displaying new dataframe (Positive = 1, Negative = 0)
train_df.head()

/tmp/ipykernel_1176/522713427.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.loc[:, 'sentiment'] = train_df['text'].apply(get_sentiment_score)


Unnamed: 0  \
0        8476   
1       10294   
2        3608   
3       10142   
4         875   

                                                                                   title  \
0                                                           You Can Smell Hillary’s Fear   
1  Watch The Exact Moment Paul Ryan Committed Political Suicide At A Trump Rally (VIDEO)   
2                                            Kerry to go to Paris in gesture of sympathy   
3   Bernie supporters on Twitter erupt in anger against the DNC: 'We tried to warn you!'   
4                                       The Battle of New York: Why This Primary Matters   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [7]:
# Importing Necessary libraries
from sklearn.naive_bayes import MultinomialNB # Naive Bayes Library
from sklearn.feature_extraction.text import CountVectorizer # Converting Text into matrix for NB

# CountVectorizer Instance
vectorizer = CountVectorizer()

# Instance of MultinomialNB
multinomial_classifier = MultinomialNB()

'''
- Preprocesses text again
- fit: learns the vocabulary from every document
- transform: Converts into a document-term matrix
- Counts the frequency of each word in individual documents
'''
x_train = vectorizer.fit_transform(train_df['text'])

# Display the shape of the matrix
print("Shape of x_train:", x_train.shape)

# Optionally, display the content of the matrix (Dense Matrix)
print("Content of x_train:\n", x_train.toarray()) 

# The target labels of the training dataframe
y_train = train_df['sentiment'] 

# (independent_features, target labels)
multinomial_classifier.fit(x_train, y_train)

# Preprocessing and transforming test data
x_test = vectorizer.transform(test_df['text'])  # Use transform() instead of fit_transform()

# Making Predictions on the test data
test_df.loc[:, 'predicted_sentiment'] = multinomial_classifier.predict(x_test)

# Displaying the new test datafram
test_df.head()

Shape of x_train: (4434, 57793)
Content of x_train:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


/tmp/ipykernel_1176/1926271544.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.loc[:, 'predicted_sentiment'] = multinomial_classifier.predict(x_test)


Unnamed: 0  \
4434        4594   
4435        5245   
4436         978   
4437        2771   
4438        8257   

                                                                                                                title  \
4434                                                                          7 things to watch for on election night   
4435                                             Trump woos women and minorities by pitting one group against another   
4436                                                       Bernie Sanders Says Bill Clinton Owes Americans an Apology   
4437                                                                   How the Iran deal might change the Middle East   
4438  HERE THEY GO AGAIN! Muslims trying to claim that the Hebrew-language Dead Sea Scrolls are Arab Muslim in origin   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      